In [ ]:
import json
import matplotlib.pyplot as plt
from PIL import Image
from types import SimpleNamespace
import random
import os
import ipywidgets as widgets
from IPython.display import display

filename = "sis/test.story-in-sequence.json"
image_folder = "images/test"
output_folder = "story_plots_test/"

with open(filename, 'r') as file:
    data = json.load(file)

images_data = data['images']
albums_data = data.get('albums', [])
annotations_data = data['annotations']

images = [json.loads(json.dumps(image), object_hook=lambda d: SimpleNamespace(**d)) for image in images_data]
albums = [json.loads(json.dumps(album), object_hook=lambda d: SimpleNamespace(**d)) for album in albums_data]

annotations = [[json.loads(json.dumps(item), object_hook=lambda d: SimpleNamespace(**d)) for item in annotation_list] for annotation_list in annotations_data]

for img in images[:5]:
    print(f"Title: {img.title}, ID: {img.id}, URL: {img.url_o}")

for album in albums[:5]:
    print(f"Description: {album.description}, ID: {album.id}, Title: {album.title}")

if annotations:
    for annotation in annotations[:5]:
        annotation = annotation[0]
        print(f"Original Text: {annotation.original_text}, Story ID: {annotation.story_id}, Photo ID: {annotation.photo_flickr_id}")

annotations_dict = {}
for annotationArray in annotations:
    for annotation in annotationArray:
        story_id = annotation.story_id
        if story_id not in annotations_dict:
            annotations_dict[story_id] = []
        annotations_dict[story_id].append(annotation)

In [ ]:
story_ids = list(annotations_dict.keys())


In [ ]:
def plot_story_images_and_annotations(story_id, save_plot=True):

    story_annotations = sorted(annotations_dict.get(story_id, []), key=lambda ann: ann.worker_arranged_photo_order)

    story_images = [img for img in images if img.id in [ann.photo_flickr_id for ann in story_annotations]]

    if story_annotations:
        story_tier = story_annotations[0].tier
        story_setting = story_annotations[0].setting
        story_id = story_annotations[0].story_id
        plot_title = f"{story_id}\nTier: {story_tier} | Setting: {story_setting}\n\n"
    else:
        plot_title = f"{story_id}\nTier: N/A | Setting: N/A"

    fig, axes = plt.subplots(nrows=len(story_images), ncols=1, figsize=(10, len(story_images) * 3))
    if len(story_images) == 1:
        axes = [axes]

    for ax, annotation in zip(axes, story_annotations):

        image = next((img for img in story_images if img.id == annotation.photo_flickr_id), None)
        
        if image:
            image_path = os.path.join(image_folder, f"{image.id}.jpg")
            if os.path.exists(image_path):
                img_data = Image.open(image_path)
                ax.imshow(img_data)
            else:
                print(f"Image file {image_path} not found.")
                continue
        else:
            ax.text(0.5, 0.5, "Image not found", horizontalalignment='center', verticalalignment='center')

        ax.axis('off')
        ax.set_title(annotation.original_text, fontsize=12)
    
    fig.suptitle(plot_title, fontsize=14)
    plt.tight_layout()

    if save_plot:
        output_path = os.path.join(output_folder, f"{story_id}.png")
        plt.savefig(output_path, bbox_inches='tight')
        print(f"Plot saved for Story ID {story_id} at {output_path}")
    
    plt.close(fig)  # Close the figure to avoid display in the notebook

# Loop through each story_id and save the plot
for story_id in story_ids:
    plot_story_images_and_annotations(story_id)